In [ ]:
# pip install mysql-connector-python
# to install mysql connector 

In [1]:
import pandas as pd
from tabulate import tabulate

In [2]:
data = pd.read_excel("C:/Users/Deepanshu/DA/Datasets/ESD.xlsx")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   EEID           1000 non-null   object        
 1   Full Name      1000 non-null   object        
 2   Job Title      1000 non-null   object        
 3   Department     1000 non-null   object        
 4   Business Unit  1000 non-null   object        
 5   Gender         1000 non-null   object        
 6   Ethnicity      1000 non-null   object        
 7   Age            1000 non-null   int64         
 8   Hire Date      1000 non-null   datetime64[ns]
 9   Annual Salary  1000 non-null   int64         
 10  Bonus %        1000 non-null   float64       
 11  Country        1000 non-null   object        
 12  City           1000 non-null   object        
 13  Exit Date      85 non-null     datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(2), object(9)
memory usage: 109.5

In [ ]:
data.duplicated('EEID').sum()

In [ ]:
# to find if there are any duplicate record for the same employee. 
# Since there can be multiple employees with same name or age or country or ethnicity, 
# I have considered multiple parameters to check there are duplicate records for the same employees
data[data.duplicated(['Full Name', 'Age', 'Ethnicity', 'Country'])]

In [ ]:
# There are no duplicate records for the same employee, however there are duplicate EEID numbers. 
# Hence We have to drop the 'EEID' and add a new column as the EEID with unique values. 
data.drop(['EEID'], axis=1, inplace=True)
# Generate Unique employee ids
unique_eeids = range(1, len(data)+1)
data['EEID'] = unique_eeids

In [ ]:
# rearranging the columns in the dataframe
data = data[['EEID', 'Full Name', 'Job Title', 'Department', 'Business Unit', 'Gender',
       'Ethnicity', 'Age', 'Hire Date', 'Annual Salary', 'Bonus %', 'Country',
       'City', 'Exit Date']]

### Connecting to a database.

In [4]:
import mysql.connector

In [5]:
conn = mysql.connector.connect(
    host = 'localhost', 
    user = 'root', 
    password = 'root123', 
    database = 'db1'
)

if conn.is_connected():
    print("Connection established !")

Connection established !


### Creating a table

In [8]:
cur = conn.cursor()

In [ ]:
query = '''
create table employees (
    EEID varchar(5) not null primary key,
    `Full Name` varchar(255) not null,
    `Job Title` varchar(255) not null,
    Department varchar(255) not null, 
    `Business Unit` VARCHAR(255) NOT NULL,
    Gender VARCHAR(10) NOT NULL,
    Ethnicity VARCHAR(50) NOT NULL,
    Age INT NOT NULL,
    `Hire Date` DATE NOT NULL,
    `Annual Salary` DECIMAL(10, 2) NOT NULL,
    `Bonus %` DECIMAL(5, 2) NOT NULL,
    Country VARCHAR(50) NOT NULL,
    City VARCHAR(50) NOT NULL,
    `Exit Date` DATE
);
'''

In [ ]:
cur.execute(query)

### Inserting records into table

In [ ]:
query = '''
insert into employees (EEID, `Full Name`, `Job Title`, Department, `Business Unit`, Gender, Ethnicity, Age, `Hire Date`, `Annual Salary`, `Bonus %`, Country, City, `Exit Date`) 
values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''

In [ ]:
for index, row in data.iterrows(): 
    cur.execute(query, (row['EEID'], row['Full Name'], row['Job Title'], row['Department'], row['Business Unit'], row['Gender'], row['Ethnicity'], row['Age'], row['Hire Date'], row['Annual Salary'], row['Bonus %'], row['Country'], row['City'], row['Exit Date']))

In [ ]:
conn.commit()

In [ ]:
for index, row in data.iterrows(): 
    print((row['EEID'], row['Full Name'], row['Job Title'], row['Department'], row['Business Unit'], row['Gender'], row['Ethnicity'], row['Age'], row['Hire Date'], row['Annual Salary'], row['Bonus %'], row['Country'], row['City'], row['Exit Date']))

### Fetching data

In [9]:
query = '''
select * from employees
'''

In [10]:
cur.execute(query)

In [11]:
result = cur.fetchall()

In [12]:
for record in result:
    print(record)

('1', 'Emily Davis', 'Sr. Manger', 'IT', 'Research & Development', 'Female', 'Black', 55, datetime.date(2016, 4, 8), Decimal('141604.00'), Decimal('0.15'), 'United States', 'Seattle', datetime.date(2021, 10, 16))
('10', 'Madeline Walker', 'Sr. Analyst', 'Finance', 'Speciality Products', 'Female', 'Caucasian', 34, datetime.date(2018, 6, 13), Decimal('77203.00'), Decimal('0.00'), 'United States', 'Chicago', datetime.date(1, 1, 1))
('100', 'Nevaeh Jones', 'Vice President', 'Sales', 'Manufacturing', 'Female', 'Caucasian', 31, datetime.date(2020, 8, 20), Decimal('219693.00'), Decimal('0.30'), 'United States', 'Austin', datetime.date(1, 1, 1))
('1000', 'Sofia Cheng', 'Vice President', 'Accounting', 'Corporate', 'Female', 'Asian', 63, datetime.date(2020, 7, 26), Decimal('216195.00'), Decimal('0.31'), 'United States', 'Miami', datetime.date(1, 1, 1))
('101', 'Samantha Adams', 'Test Engineer', 'Engineering', 'Research & Development', 'Female', 'Caucasian', 45, datetime.date(2013, 4, 22), Decima

# Exploratory Data Analysis using SQL

##### Q. How many employees are represented in the database? 

In [13]:
query = '''
select count(EEID) from employees;
'''

In [14]:
cur.execute(query)

In [15]:
result = cur.fetchall()

In [16]:
print(result)

[(1000,)]


##### Q. What is the distribution of genders across the employees? 

In [17]:
query = '''
select 
	gender as Gender, 
    count(EEID) as Gender_Count
from db1.employees
group by gender;
'''

In [18]:
cur.execute(query)

In [19]:
result = cur.fetchall()

In [20]:
print(result)

[('Female', 518), ('Male', 482)]


In [21]:
# python function display result in grid format
def print_result(result):
    # get the column names
    column_names = [column[0] for column in cur.description]

    # print the result as table
    print(tabulate(result, headers=column_names, tablefmt='grid'))

In [22]:
print_result(result)

+----------+----------------+
| Gender   |   Gender_Count |
+==========+================+
| Female   |            518 |
+----------+----------------+
| Male     |            482 |
+----------+----------------+


##### Q. What are the common job titles within the company?

In [23]:
query = '''
select 
    distinct `job title` as `Job Title`, 
    count(EEID) over (partition by `Job Title`) as `Count of Job Title`
from employees 
order by `Count of Job Title` desc
limit 0, 5;
'''

In [24]:
cur.execute(query)
result = cur.fetchall()
print_result(result)

+----------------+----------------------+
| Job Title      |   Count of Job Title |
+================+======================+
| Director       |                  121 |
+----------------+----------------------+
| Sr. Manger     |                  110 |
+----------------+----------------------+
| Vice President |                  105 |
+----------------+----------------------+
| Manager        |                   98 |
+----------------+----------------------+
| Sr. Analyst    |                   70 |
+----------------+----------------------+


##### Q. What is the average age of employees within the company? 

In [ ]:
query = '''
select avg(age) from employees; 
'''

In [ ]:
cur.execute(query)
result = cur.fetchall()
print_result(result)

##### Q. What is the distribution of enthnicities among the employees?

In [ ]:
query = '''
select 
    distinct Ethnicity, 
    Gender, 
    count(EEID) over (partition by Ethnicity, Gender) as Count
from employees;
'''

In [ ]:
cur.execute(query)
result = cur.fetchall()
print_result(result)

##### Q. How many employees do each department have? 

In [ ]:
query = '''
select 
    Department, 
    count(EEID) as `Count of Employees`
from employees 
group by Department 
order by `Count of Employees` desc;
'''

In [ ]:
cur.execute(query)
result = cur.fetchall()
print_result(result)

##### Q. What is the average annual salary of employees in each department? 

In [47]:
query = '''
select 
    Department, 
    avg(`Annual Salary`) as `Average Salary` 
from employees 
group by Department 
order by `Average Salary` desc;
'''

In [48]:
cur.execute(query)
result = cur.fetchall()
print_result(result)

+-----------------+------------------+
| Department      |   Average Salary |
+=================+==================+
| Marketing       |         129663   |
+-----------------+------------------+
| Accounting      |         123147   |
+-----------------+------------------+
| Finance         |         122803   |
+-----------------+------------------+
| Human Resources |         118058   |
+-----------------+------------------+
| Sales           |         111050   |
+-----------------+------------------+
| Engineering     |         109035   |
+-----------------+------------------+
| IT              |          97790.5 |
+-----------------+------------------+


##### Q. What is the average age of employees in each department?

In [27]:
query = '''
select 
    Department, 
    avg(Age) as 'Average Age'
from employees 
group by Department;
'''

In [28]:
cur.execute(query)
result = cur.fetchall()
print_result(result)

+-----------------+---------------+
| Department      |   Average Age |
+=================+===============+
| IT              |       44.3444 |
+-----------------+---------------+
| Finance         |       45.2917 |
+-----------------+---------------+
| Sales           |       43.6429 |
+-----------------+---------------+
| Accounting      |       43.6563 |
+-----------------+---------------+
| Engineering     |       45.6709 |
+-----------------+---------------+
| Human Resources |       44.456  |
+-----------------+---------------+
| Marketing       |       43.2167 |
+-----------------+---------------+


##### Q. Break down of employee ages. 

In [45]:
query ='''
select 
    Department, 
    sum(case when age between 20 and 29 then 1 else 0 end) as `20-29`,
    sum(case when age between 30 and 39 then 1 else 0 end) as `30-39`,
    sum(case when age between 40 and 49 then 1 else 0 end) as `40-49`,
    sum(case when age between 50 and 59 then 1 else 0 end) as `50-59`,
    sum(case when age between 60 and 69 then 1 else 0 end) as `60-69`, 
    count(EEID) as `Total Count`
from employees
group by Department; 
'''

In [46]:
cur.execute(query)
result = cur.fetchall()
print_result(result)

+-----------------+---------+---------+---------+---------+---------+---------------+
| Department      |   20-29 |   30-39 |   40-49 |   50-59 |   60-69 |   Total Count |
+=================+=========+=========+=========+=========+=========+===============+
| IT              |      29 |      55 |      72 |      66 |      19 |           241 |
+-----------------+---------+---------+---------+---------+---------+---------------+
| Finance         |      12 |      33 |      25 |      30 |      20 |           120 |
+-----------------+---------+---------+---------+---------+---------+---------------+
| Sales           |      18 |      36 |      41 |      29 |      16 |           140 |
+-----------------+---------+---------+---------+---------+---------+---------------+
| Accounting      |      16 |      21 |      25 |      26 |       8 |            96 |
+-----------------+---------+---------+---------+---------+---------+---------------+
| Engineering     |      17 |      30 |      48 |     